# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [403]:
import pandas as pd
import boto3
import sagemaker

In [404]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [405]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/plagiarism_detection'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [406]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

pytorch-inference-2020-04-05-11-19-22-723/model.tar.gz
pytorch-inference-2020-04-05-11-40-46-909/model.tar.gz
pytorch-inference-2020-04-05-12-04-43-246/model.tar.gz
pytorch-inference-2020-04-05-12-19-08-095/model.tar.gz
pytorch-inference-2020-04-05-13-35-48-624/model.tar.gz
pytorch-inference-2020-04-05-16-55-53-115/model.tar.gz
pytorch-inference-2020-04-05-18-51-50-594/model.tar.gz
pytorch-inference-2020-04-05-19-21-13-965/model.tar.gz
pytorch-training-2020-04-05-11-13-37-925/debug-output/collections/000000000/worker_0_collections.json
pytorch-training-2020-04-05-11-13-37-925/debug-output/events/000000000000/000000000000_worker_0.tfevents
pytorch-training-2020-04-05-11-13-37-925/debug-output/events/000000000500/000000000500_worker_0.tfevents
pytorch-training-2020-04-05-11-13-37-925/debug-output/index/000000000/000000000000_worker_0.json
pytorch-training-2020-04-05-11-13-37-925/debug-output/index/000000000/000000000500_worker_0.json
pytorch-training-2020-04-05-11-13-37-925/debug-output/

---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

### Start of Custom Model

In [407]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import os

In [408]:
EPOCHS = 100
BATCH_SIZE = 10
LEARNING_RATE = 0.001
RANDOM_SEED = 1000

torch.manual_seed(RANDOM_SEED)

In [409]:
# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None, names=None)

# Turn the input pandas dataframe into tensors
y_train = torch.from_numpy(train_sample[[0]].values).float().squeeze()
X_train = torch.from_numpy(train_sample.drop([0], axis=1).values).float()

y_test = torch.from_numpy(test_sample[[0]].values).float().squeeze()
X_test = torch.from_numpy(test_sample.drop([0], axis=1).values).float()

In [410]:
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(X_test))

In [411]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [492]:
# torch imports
import torch.nn.functional as F
import torch.nn as nn
import torch


## TODO: Complete this classifier
class BinaryClassifier(nn.Module):
    """
    Define a neural network that performs binary classification.
    The network should accept your number of features as input, and produce 
    a single sigmoid value, that can be rounded to a label: 0 or 1, as output.
    
    Notes on training:
    To train a binary classifier in PyTorch, use BCELoss.
    BCELoss is binary cross entropy loss, documentation: https://pytorch.org/docs/stable/nn.html#torch.nn.BCELoss
    """

    ## TODO: Define the init function, the input params are required (for loading code in train.py to work)
    def __init__(self, input_features, hidden_dim, output_dim, dropout=0.1):
        """
        Initialize the model by setting up linear layers.
        Use the input parameters to help define the layers of your model.
        :param input_features: the number of input features in your training/test data
        :param hidden_dim: helps define the number of nodes in the hidden layer(s)
        :param output_dim: the number of outputs you want to produce
        """
        super(BinaryClassifier, self).__init__()

        # define any initial layers, here
        # number of input features is 3
        self.layer_1 = nn.Linear(input_features, hidden_dim)
        self.layer_2 = nn.Linear(hidden_dim, hidden_dim)
        self.layer_out = nn.Linear(hidden_dim, output_dim)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)

    
    ## TODO: Define the feedforward behavior of the network
    def forward(self, x):
        """
        Perform a forward pass of our model on input features, x.
        :param x: A batch of input features of size (batch_size, input_features)
        :return: A single, sigmoid-activated value as output
        """
        
        # define the feedforward behavior
        x = x.float()
        x = self.relu(self.layer_1(x))
        x = self.relu(self.layer_2(x))
        x = self.dropout(x)
        x = self.layer_out(x)

        return torch.sigmoid(x)
        
    

In [493]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
###################### OUTPUT ######################
cuda:0

cpu


In [506]:
model = BinaryClassifier(3, 32, 1)
model.to(device)
print(model)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

BinaryClassifier(
  (layer_1): Linear(in_features=3, out_features=32, bias=True)
  (layer_2): Linear(in_features=32, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [507]:
def calculate_accuracy(y_pred, y_test):
    y_pred_tag = torch.round(y_pred)

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [508]:
# # training loop is provided
model.train() # Make sure that the model is in training mode.
for epoch in range(1, EPOCHS + 1):
    epoch_loss = 0
    epoch_accuracy = 0

    for batch in train_loader:
        # get data
        batch_x, batch_y = batch

        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()

        # get predictions from model
        pred_y = model(batch_x)

        # perform backprop
        loss = criterion(pred_y, batch_y.unsqueeze(1))
        accuracy = calculate_accuracy(pred_y, batch_y.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_accuracy += accuracy.item()

    print(f'Epoch {epoch+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_accuracy/len(train_loader):.3f}')

Epoch 001: | Loss: 0.67041 | Acc: 60.000
Epoch 002: | Loss: 0.66294 | Acc: 60.000
Epoch 003: | Loss: 0.65545 | Acc: 60.000
Epoch 004: | Loss: 0.65161 | Acc: 60.000
Epoch 005: | Loss: 0.64617 | Acc: 60.000
Epoch 006: | Loss: 0.64098 | Acc: 60.000
Epoch 007: | Loss: 0.63358 | Acc: 60.000
Epoch 008: | Loss: 0.62734 | Acc: 60.000
Epoch 009: | Loss: 0.62027 | Acc: 60.000
Epoch 010: | Loss: 0.60950 | Acc: 60.000
Epoch 011: | Loss: 0.59949 | Acc: 60.000
Epoch 012: | Loss: 0.58683 | Acc: 60.000
Epoch 013: | Loss: 0.57615 | Acc: 60.000
Epoch 014: | Loss: 0.55830 | Acc: 60.000
Epoch 015: | Loss: 0.53831 | Acc: 61.429
Epoch 016: | Loss: 0.52263 | Acc: 68.571
Epoch 017: | Loss: 0.50528 | Acc: 68.571
Epoch 018: | Loss: 0.49457 | Acc: 70.000
Epoch 019: | Loss: 0.47244 | Acc: 78.571
Epoch 020: | Loss: 0.45031 | Acc: 88.571
Epoch 021: | Loss: 0.43384 | Acc: 90.000
Epoch 022: | Loss: 0.41164 | Acc: 91.429
Epoch 023: | Loss: 0.39725 | Acc: 94.286
Epoch 024: | Loss: 0.38698 | Acc: 94.286
Epoch 025: | Los

In [511]:
model.eval()
# Compute the result of applying the model to the input data
# The variable `out_label` should be a rounded value, either 1 or 0
out = model(X_test)
out_np = out.cpu().detach().numpy()
out_label = out_np.round()
print(out_label)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]


In [512]:
print(confusion_matrix(y_test, out_label))

[[10  0]
 [ 0 15]]


In [514]:
print(classification_report(y_test, out_label))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      1.00      1.00        15

   micro avg       1.00      1.00      1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25



### End of Custom Model

In [515]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset, DataLoader

# imports the model in model.py by name
from model import BinaryClassifier

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    #Set default tensor type, then move model and data to GPU
    if torch.cuda.is_available():
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        torch.set_default_tensor_type('torch.FloatTensor')
    
    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_avai

                        help='output dim (default: 1)')
    
    # args holds all passed-in arguments
    args = parser.parse_args()

    #Set default tensor type, then move model and data to GPU
    if torch.cuda.is_available():
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        torch.set_default_tensor_type('torch.FloatTensor')
        
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device {}.".format(device))

    torch.manual_seed(args.seed)

    # Load the training data.
    train_loader = _get_train_data_loader(args.batch_size, args.data_dir)


    ## --- Your code here --- ##
    
    ## TODO:  Build the model by passing in the input params
    # To get params from the parser, call args.argument_name, ex. args.epochs or ards.hidden_dim
    # Don't forget to move your model .to(device) to move to GPU , if appropriate
    model = BinaryClassifier(args.input_features, args.hidden_dim, args.output_dim)
    mo

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [516]:

# your import and estimator code, here
from sagemaker.pytorch import PyTorch

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [517]:
!pip install nvidia-ml-py3
!pip install pandas

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [519]:
%%time

# Train your estimator on S3 training data
estimator = PyTorch(entry_point="train.py",
                    source_dir="source_pytorch",
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 100,
                        'batch-size': 10,
                        'seed': 1000,
                        'input_features': 3,
                        'hidden_dim': 32,
                        'output_dim': 1
                    })

estimator.fit({'train': input_data})


2020-04-06 13:31:38 Starting - Starting the training job...
2020-04-06 13:31:40 Starting - Launching requested ML instances......
2020-04-06 13:32:45 Starting - Preparing the instances for training......
2020-04-06 13:33:57 Downloading - Downloading input data...
2020-04-06 13:34:23 Training - Downloading the training image..............bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-04-06 13:36:48,611 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-04-06 13:36:48,638 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-04-06 13:36:48,642 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-04-06 13:36:48,959 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-04-06 13:36:48,960 sagemaker-containers INFO     Generating setup.cfg
2020-04-06 13:3


2020-04-06 13:37:11 Completed - Training job completed
Training seconds: 194
Billable seconds: 194
CPU times: user 850 ms, sys: 44.2 ms, total: 894 ms
Wall time: 6min 13s


## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [520]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.4.0',
                     entry_point='predict.py',
                     source_dir="source_pytorch")

# deploy your model to create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')


---------------------!CPU times: user 632 ms, sys: 44.5 ms, total: 677 ms
Wall time: 10min 33s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [521]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [522]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x.values)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [526]:
# Second: calculate the test accuracy
accuracy = sum([bool(test_y_preds[i] == test_y[i]) for i in range(len(test_y))]) / len(test_y)

print('Accuracy:')
print(accuracy)
print()

print('Confusion Matrix:')
print(confusion_matrix(test_y_preds, test_y))
print()

print('Classification Report: ')
print(classification_report(test_y_preds, test_y))


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

Accuracy:
1.0

Confusion Matrix:
[[10  0]
 [ 0 15]]

Classification Report: 
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      1.00      1.00        15

   micro avg       1.00      1.00      1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25


Predicted class labels: 
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

** Answer**: 
As you can see on the confusion matrix, there are no false positives and false negatives produced by the model. It is due to the selection of best predictors, finding the compromise on which predictors to choose and noting that the most accurate model that best generalizes the ground truth can be achieved through simplistic models. As you can see, the model only has one hidden layer with the same shape for both inputs and outputs and dropout layer to prevent overfitting. And since I also ran it in 100 epochs, I made sure that it saturates to the best model.

### Question 2: How did you decide on the type of model to use? 

** Answer**:

Like I said, it is a combination of careful selection of covariates to use as well as the simplistic model with high regard to the prevention of overfitting especially when we have a limited number of predictors (see above). Also, as an extra challenge, I used PyTorch instead of the built-in linear learners provided by Amazon SageMaker.

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [527]:
# uncomment and fill in the line below!
predictor.delete_endpoint()


### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [528]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '91607C5724214959',
   'HostId': 'RB3z8NjiAUwgb7APW/+sqmIYUbOcGS9KbaflVAR/Kj5iL7INDgwi+m7R/kiTYaTcR99X3rXQYpY=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'RB3z8NjiAUwgb7APW/+sqmIYUbOcGS9KbaflVAR/Kj5iL7INDgwi+m7R/kiTYaTcR99X3rXQYpY=',
    'x-amz-request-id': '91607C5724214959',
    'date': 'Mon, 06 Apr 2020 14:02:16 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'pytorch-training-2020-04-05-13-16-30-134/debug-output/collections/000000000/worker_0_collections.json'},
   {'Key': 'pytorch-training-2020-04-05-16-35-13-932/source/sourcedir.tar.gz'},
   {'Key': 'pytorch-training-2020-04-05-13-16-30-134/debug-output/training_job_end.ts'},
   {'Key': 'pytorch-training-2020-04-05-16-41-04-793/debug-output/events/000000000000/000000000000_worker_0.tfevents'},
   {'Key': 'pytorch-training-2020-04-05-16-47-03-921/

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!